# Lab 4: Prefix Tuning - Deployment Guide

## 🎯 實驗目標

本 Notebook 探討 **Prefix Tuning 在生產環境中的部署策略**。與 LoRA 和 IA³ 等可完全合併的方法不同,Prefix Tuning 的前綴參數**無法直接合併到基礎模型中**,因此需要特殊的部署方案。

### 關鍵學習要點
- 理解 Prefix Tuning 無法合併的技術原因
- 掌握前綴參數的優化與壓縮技術
- 實現高效的 Adapter 載入與推理部署
- 分析不同部署場景下的最佳實踐
- 學習多任務前綴的管理策略

---

## 1. Prefix Tuning 部署特性分析

### 1.1 為什麼 Prefix Tuning 無法合併?

**技術原因**:
- **結構性修改**: Prefix 在每一層的 Key/Value 都注入新的向量,改變了注意力機制的計算結構
- **動態注入**: Prefix 向量在推理時動態拼接到原始序列,無法靜態融合到權重矩陣
- **位置依賴**: Prefix 位置對模型行為有顯著影響,合併會破壞這種位置關係

**與可合併方法的對比**:

| 方法 | 是否可合併 | 原理 | 推理開銷 |
|:---|:---|:---|:---|
| **LoRA** | ✅ 可合併 | `W' = W + B·A` 矩陣加法 | 零開銷 |
| **IA³** | ✅ 可合併 | `W' = W ⊙ scale` 元素乘法 | 零開銷 |
| **Prefix Tuning** | ❌ **不可合併** | 動態 KV 注入 | **有開銷** |
| **Prompt Tuning** | ❌ 不可合併 | 輸入序列拼接 | 有開銷 |
| **Adapter** | ❌ 不可合併 | 新增模組層 | 有開銷 |

---

## 2. 環境準備與模型載入

In [ ]:
# 導入必要的庫
import torch
import torch.nn as nn
import time
import gc
import os
import json
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    pipeline
)
from peft import (
    PeftModel, 
    PrefixTuningConfig,
    get_peft_model,
    TaskType
)

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用設備: {device}")

# 設定隨機種子
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

### 2.1 載入訓練好的 Prefix Tuning 模型

In [ ]:
# 模型路徑設定
base_model_name = "gpt2"
adapter_path = "./gpt2-prefix-tuning-imdb"  # 假設這是訓練好的 adapter 路徑

# 載入基礎模型和 tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto" if torch.cuda.is_available() else None
)

print(f"基礎模型參數量: {base_model.num_parameters():,}")
print(f"基礎模型記憶體佔用: {base_model.get_memory_footprint() / 1024**2:.2f} MB")

In [ ]:
# 載入 Prefix Tuning adapter (如果存在)
if os.path.exists(adapter_path):
    # 找到最新的 checkpoint
    checkpoints = [d for d in os.listdir(adapter_path) if d.startswith("checkpoint-")]
    if checkpoints:
        latest_checkpoint = max(
            [os.path.join(adapter_path, d) for d in checkpoints],
            key=os.path.getmtime
        )
        peft_model = PeftModel.from_pretrained(base_model, latest_checkpoint)
        print(f"成功載入 Prefix Tuning adapter: {latest_checkpoint}")
    else:
        peft_model = PeftModel.from_pretrained(base_model, adapter_path)
        print(f"成功載入 Prefix Tuning adapter: {adapter_path}")
else:
    # 如果沒有訓練好的 adapter,創建示範配置
    print("未找到訓練好的 adapter,創建示範 Prefix Tuning 配置...")
    
    prefix_config = PrefixTuningConfig(
        task_type=TaskType.CAUSAL_LM,
        num_virtual_tokens=20,
        prefix_projection=True,
        encoder_hidden_size=768
    )
    
    peft_model = get_peft_model(base_model, prefix_config)
    print("創建了示範 Prefix Tuning 模型")

# 顯示可訓練參數統計
peft_model.print_trainable_parameters()

## 3. Prefix Tuning 結構分析

### 3.1 前綴參數結構探索

In [ ]:
def analyze_prefix_structure(model):
    """分析 Prefix Tuning 的結構和參數"""
    prefix_params = {}
    total_prefix_params = 0
    
    for name, param in model.named_parameters():
        if 'prefix' in name.lower() or 'prompt' in name.lower():
            prefix_params[name] = {
                'shape': param.shape,
                'dtype': param.dtype,
                'requires_grad': param.requires_grad,
                'device': param.device,
                'num_params': param.numel()
            }
            total_prefix_params += param.numel()
            
            print(f"Prefix 參數: {name}")
            print(f"  形狀: {param.shape}")
            print(f"  數據類型: {param.dtype}")
            print(f"  參數量: {param.numel():,}")
            print()
    
    print(f"總 Prefix 參數量: {total_prefix_params:,}")
    print(f"佔基礎模型參數比例: {total_prefix_params / base_model.num_parameters() * 100:.4f}%")
    
    return prefix_params

prefix_structure = analyze_prefix_structure(peft_model)

### 3.2 前綴參數記憶體分析

In [ ]:
def calculate_prefix_memory(prefix_params):
    """計算前綴參數的記憶體佔用"""
    total_memory = 0
    
    for name, info in prefix_params.items():
        # 根據數據類型計算記憶體
        dtype_size = {
            torch.float32: 4,
            torch.float16: 2,
            torch.bfloat16: 2,
            torch.int32: 4,
            torch.int64: 8
        }
        
        size_bytes = info['num_params'] * dtype_size.get(info['dtype'], 4)
        total_memory += size_bytes
        
        print(f"{name}: {size_bytes / 1024:.2f} KB")
    
    print(f"\n總前綴參數記憶體: {total_memory / 1024:.2f} KB ({total_memory / 1024**2:.4f} MB)")
    return total_memory

prefix_memory = calculate_prefix_memory(prefix_structure)

## 4. 推理性能基準測試

### 4.1 性能測試函數

In [ ]:
def benchmark_inference(model, tokenizer, test_prompts, num_runs=5):
    """推理性能基準測試"""
    model.eval()
    total_time = 0
    results = []
    
    with torch.no_grad():
        for run in range(num_runs):
            start_time = time.time()
            
            for prompt in test_prompts:
                inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
                if torch.cuda.is_available():
                    inputs = {k: v.cuda() for k, v in inputs.items()}
                
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=50,
                    do_sample=True,
                    temperature=0.7,
                    top_k=50,
                    top_p=0.95,
                    pad_token_id=tokenizer.eos_token_id
                )
                
                generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                if run == 0:  # 只在第一輪保存結果
                    results.append({
                        'prompt': prompt,
                        'generated': generated_text[len(prompt):].strip()
                    })
            
            end_time = time.time()
            total_time += (end_time - start_time)
    
    avg_time = total_time / num_runs
    return avg_time, results

### 4.2 執行推理測試

In [ ]:
# 測試提示詞
test_prompts = [
    "This movie was absolutely",
    "I think this film was",
    "The acting in this movie",
    "Overall, my impression is"
]

print("=== 帶 Prefix 的推理性能測試 ===")
prefix_time, prefix_results = benchmark_inference(peft_model, tokenizer, test_prompts)
print(f"平均推理時間: {prefix_time:.4f} 秒")
print(f"記憶體使用: {peft_model.get_memory_footprint() / 1024**2:.2f} MB")

print("\n=== 生成示例 ===")
for i, result in enumerate(prefix_results[:2]):  # 只顯示前兩個示例
    print(f"提示詞 {i+1}: {result['prompt']}")
    print(f"生成: {result['generated']}")
    print()

### 4.3 與基礎模型對比

In [ ]:
print("=== 基礎模型推理性能測試 ===")
base_time, base_results = benchmark_inference(base_model, tokenizer, test_prompts)
print(f"平均推理時間: {base_time:.4f} 秒")
print(f"記憶體使用: {base_model.get_memory_footprint() / 1024**2:.2f} MB")

# 計算性能開銷
time_overhead = (prefix_time - base_time) / base_time * 100
memory_overhead = (peft_model.get_memory_footprint() - base_model.get_memory_footprint()) / base_model.get_memory_footprint() * 100

print("\n=== 性能開銷分析 ===")
print(f"推理時間增加: {time_overhead:.2f}%")
print(f"記憶體使用增加: {memory_overhead:.2f}%")

print("\n=== 基礎模型生成示例 ===")
for i, result in enumerate(base_results[:2]):
    print(f"提示詞 {i+1}: {result['prompt']}")
    print(f"生成: {result['generated']}")
    print()

## 5. Prefix 參數優化與壓縮

### 5.1 移除訓練時的 MLP 重參數化層

In [ ]:
def optimize_prefix_for_inference(model):
    """
    優化 Prefix Tuning 模型以進行推理
    在訓練階段,使用 MLP 重參數化來穩定訓練
    在推理階段,可以移除 MLP,直接使用預計算的 prefix
    """
    print("=== Prefix 推理優化 ===")
    print("注意: 此步驟通常在保存模型時自動完成")
    print("PEFT 庫會自動處理 MLP 的移除和 prefix 的預計算")
    
    # 顯示當前模型結構
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n總參數量: {total_params:,}")
    print(f"可訓練參數量: {trainable_params:,}")
    
    return model

optimized_model = optimize_prefix_for_inference(peft_model)

### 5.2 Prefix 參數量化

In [ ]:
def quantize_prefix_parameters(model, target_dtype=torch.float16):
    """
    量化前綴參數以減少記憶體佔用
    注意: 這會修改模型,請確保已保存原始版本
    """
    print(f"=== 量化 Prefix 參數到 {target_dtype} ===")
    
    quantized_params = 0
    original_memory = 0
    quantized_memory = 0
    
    for name, param in model.named_parameters():
        if 'prefix' in name.lower() or 'prompt' in name.lower():
            original_dtype = param.dtype
            original_size = param.numel() * param.element_size()
            
            # 量化參數
            param.data = param.data.to(target_dtype)
            
            quantized_size = param.numel() * param.element_size()
            quantized_params += 1
            original_memory += original_size
            quantized_memory += quantized_size
            
            print(f"量化 {name}: {original_dtype} -> {target_dtype}")
            print(f"  記憶體: {original_size / 1024:.2f} KB -> {quantized_size / 1024:.2f} KB")
    
    compression_ratio = (original_memory - quantized_memory) / original_memory * 100
    print(f"\n量化了 {quantized_params} 個 prefix 參數")
    print(f"記憶體壓縮率: {compression_ratio:.2f}%")
    print(f"節省記憶體: {(original_memory - quantized_memory) / 1024:.2f} KB")
    
    return model

# 演示量化 (實際使用時請謹慎,可能影響性能)
print("注意: 量化會修改模型,這裡僅作演示")
print("實際部署時請先在驗證集上測試量化後的性能\n")

## 6. 模型保存與部署

### 6.1 保存 Prefix Tuning Adapter

In [ ]:
# 設定保存路徑
save_path = "./gpt2-prefix-tuning-deployed"

print(f"=== 保存 Prefix Tuning Adapter 到: {save_path} ===")

# 保存 PEFT adapter (只保存前綴參數)
peft_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Adapter 保存成功!")

# 檢查保存的文件
saved_files = os.listdir(save_path)
print(f"\n保存的文件: {saved_files}")

# 計算 adapter 大小
adapter_size = 0
for file in saved_files:
    file_path = os.path.join(save_path, file)
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path)
        adapter_size += size
        print(f"  {file}: {size / 1024:.2f} KB")

print(f"\nAdapter 總大小: {adapter_size / 1024:.2f} KB ({adapter_size / 1024**2:.4f} MB)")

### 6.2 驗證保存的模型

In [ ]:
print("=== 驗證保存的 Adapter ===")

# 重新載入基礎模型
fresh_base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto" if torch.cuda.is_available() else None
)

# 從保存路徑載入 adapter
reloaded_model = PeftModel.from_pretrained(fresh_base_model, save_path)
reloaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

print(f"✅ 成功載入保存的 adapter")

# 快速測試
test_prompt = "This movie is absolutely"
inputs = reloaded_tokenizer(test_prompt, return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}

with torch.no_grad():
    outputs = reloaded_model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,
        temperature=0.7,
        pad_token_id=reloaded_tokenizer.eos_token_id
    )

generated_text = reloaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\n測試生成:")
print(f"提示詞: {test_prompt}")
print(f"生成: {generated_text[len(test_prompt):].strip()}")
print("\n✅ 保存的 adapter 運作正常!")

## 7. 生產環境部署策略

### 7.1 部署架構選擇

In [ ]:
deployment_strategies = {
    "單任務部署": {
        "描述": "一個服務只服務一個任務",
        "優點": [
            "部署簡單",
            "推理速度最快",
            "資源隔離"
        ],
        "缺點": [
            "資源利用率低",
            "需要多個模型實例"
        ],
        "適用場景": "單一任務、高QPS需求"
    },
    "多任務共享基礎模型": {
        "描述": "共享基礎模型,動態切換 prefix",
        "優點": [
            "記憶體效率高",
            "支持多任務",
            "prefix 切換開銷小"
        ],
        "缺點": [
            "需要 prefix 管理邏輯",
            "批次處理複雜"
        ],
        "適用場景": "多任務、中等QPS"
    },
    "批次推理優化": {
        "描述": "相同任務批次處理",
        "優點": [
            "吞吐量最大化",
            "GPU 利用率高"
        ],
        "缺點": [
            "延遲增加",
            "需要請求排隊"
        ],
        "適用場景": "離線推理、高吞吐量需求"
    }
}

print("=== Prefix Tuning 部署策略對比 ===")
for strategy, details in deployment_strategies.items():
    print(f"\n【{strategy}】")
    print(f"描述: {details['描述']}")
    print(f"優點: {', '.join(details['優點'])}")
    print(f"缺點: {', '.join(details['缺點'])}")
    print(f"適用場景: {details['適用場景']}")

### 7.2 多任務 Prefix 管理示例

In [ ]:
class MultiTaskPrefixManager:
    """
    多任務 Prefix 管理器
    用於在生產環境中管理和切換不同任務的 prefix
    """
    def __init__(self, base_model, tokenizer):
        self.base_model = base_model
        self.tokenizer = tokenizer
        self.task_adapters = {}  # 任務名稱 -> adapter 路徑
        self.current_task = None
        self.current_model = None
        
    def register_task(self, task_name, adapter_path):
        """註冊一個任務的 adapter"""
        self.task_adapters[task_name] = adapter_path
        print(f"註冊任務: {task_name} -> {adapter_path}")
        
    def switch_task(self, task_name):
        """切換到指定任務"""
        if task_name not in self.task_adapters:
            raise ValueError(f"未知任務: {task_name}")
            
        if task_name == self.current_task:
            print(f"已經在任務 {task_name} 上,無需切換")
            return
            
        print(f"切換到任務: {task_name}")
        adapter_path = self.task_adapters[task_name]
        
        # 載入對應的 adapter
        self.current_model = PeftModel.from_pretrained(
            self.base_model, 
            adapter_path
        )
        self.current_task = task_name
        
    def generate(self, prompt, task_name=None, **generate_kwargs):
        """使用指定任務的 prefix 生成文本"""
        if task_name:
            self.switch_task(task_name)
            
        if self.current_model is None:
            raise RuntimeError("請先切換到一個任務")
            
        inputs = self.tokenizer(prompt, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
            
        with torch.no_grad():
            outputs = self.current_model.generate(
                **inputs,
                **generate_kwargs
            )
            
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    def list_tasks(self):
        """列出所有已註冊的任務"""
        return list(self.task_adapters.keys())

# 演示使用
print("=== 多任務 Prefix 管理器示例 ===")
print("\n這是一個管理多個任務 prefix 的框架")
print("在生產環境中,可以預先載入多個任務的 adapter")
print("根據請求動態切換,無需重新載入整個模型")

### 7.3 生產環境部署配置

In [ ]:
# 創建部署配置文件
deployment_config = {
    "model_info": {
        "base_model": base_model_name,
        "peft_method": "Prefix Tuning",
        "num_virtual_tokens": 20,
        "mergeable": False,
        "adapter_size_mb": adapter_size / 1024**2
    },
    "performance_metrics": {
        "inference_time_seconds": prefix_time,
        "vs_base_model_overhead_percent": time_overhead,
        "memory_overhead_percent": memory_overhead,
        "adapter_loading_time_seconds": "< 1s (estimated)"
    },
    "deployment_requirements": {
        "python_packages": [
            "torch>=1.9.0",
            "transformers>=4.20.0",
            "peft>=0.3.0",
            "tokenizers>=0.12.0"
        ],
        "minimum_gpu_memory_gb": 4,
        "recommended_gpu_memory_gb": 8
    },
    "deployment_notes": {
        "adapter_management": "需要同時管理基礎模型和 adapter",
        "inference_overhead": "存在推理開銷,不可消除",
        "multi_task_support": "可以共享基礎模型,動態切換 adapter",
        "scalability": "適合多任務場景,單任務建議考慮 LoRA 或 IA³"
    },
    "usage_example": {
        "load_base_model": f"base_model = AutoModelForCausalLM.from_pretrained('{base_model_name}')",
        "load_adapter": f"model = PeftModel.from_pretrained(base_model, '{save_path}')",
        "inference": "model.generate(...)"
    },
    "optimization_tips": [
        "使用 FP16 或 BF16 減少記憶體佔用",
        "相同任務的請求批次處理",
        "預載入常用任務的 adapter",
        "使用 KV Cache 加速生成",
        "考慮使用 vLLM 等推理引擎優化"
    ]
}

# 保存配置文件
config_path = os.path.join(save_path, "deployment_config.json")
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(deployment_config, f, indent=2, ensure_ascii=False)

print("=== 生產環境部署配置 ===")
print(json.dumps(deployment_config, indent=2, ensure_ascii=False))
print(f"\n配置文件已保存到: {config_path}")

## 8. Prefix Tuning 與其他 PEFT 方法的部署對比

### 8.1 部署特性對比矩陣

In [ ]:
peft_deployment_comparison = {
    "LoRA": {
        "可合併性": "✅ 可完全合併",
        "推理開銷": "零開銷 (合併後)",
        "部署複雜度": "低 (單一模型文件)",
        "多任務支持": "需要多個合併後的模型",
        "適用場景": "單任務、追求極致性能"
    },
    "IA³": {
        "可合併性": "✅ 可完全合併",
        "推理開銷": "零開銷 (合併後)",
        "部署複雜度": "低 (單一模型文件)",
        "多任務支持": "需要多個合併後的模型",
        "適用場景": "極致效率、資源受限"
    },
    "Prefix Tuning": {
        "可合併性": "❌ 不可合併",
        "推理開銷": "有開銷 (KV 注入)",
        "部署複雜度": "中 (基礎模型 + adapter)",
        "多任務支持": "✅ 共享基礎模型",
        "適用場景": "多任務、靈活切換"
    },
    "Prompt Tuning": {
        "可合併性": "❌ 不可合併",
        "推理開銷": "有開銷 (輸入拼接)",
        "部署複雜度": "中 (基礎模型 + prompts)",
        "多任務支持": "✅ 共享基礎模型",
        "適用場景": "超大模型、極簡 adapter"
    },
    "Adapter Layers": {
        "可合併性": "❌ 不可合併",
        "推理開銷": "有開銷 (額外層)",
        "部署複雜度": "高 (模型結構修改)",
        "多任務支持": "✅ 模組化切換",
        "適用場景": "NLU 任務、模組化設計"
    }
}

print("=== PEFT 方法部署特性對比 ===")
print(f"{'方法':<15} {'可合併性':<15} {'推理開銷':<20} {'部署複雜度':<25} {'多任務支持':<20}")
print("-" * 100)

for method, features in peft_deployment_comparison.items():
    print(f"{method:<15} {features['可合併性']:<15} {features['推理開銷']:<20} {features['部署複雜度']:<25} {features['多任務支持']:<20}")

print("\n=== Prefix Tuning 部署定位 ===")
print("✅ 優勢: 多任務場景下記憶體效率高")
print("✅ 優勢: Adapter 切換靈活")
print("⚠️  劣勢: 存在推理開銷")
print("⚠️  劣勢: 部署相對複雜")
print("\n💡 建議: 單任務高性能場景選擇 LoRA/IA³,多任務靈活場景選擇 Prefix Tuning")

## 9. 部署最佳實踐與建議

### 9.1 部署檢查清單

In [ ]:
print("=== Prefix Tuning 部署檢查清單 ===")
print("")
print("📋 部署前準備:")
print("  □ 確認 Prefix Tuning adapter 訓練完成並保存")
print("  □ 在驗證集上驗證 adapter 性能")
print("  □ 記錄基準性能指標 (延遲、吞吐量)")
print("  □ 確定部署架構 (單任務 vs 多任務)")
print("")
print("🔧 部署配置:")
print("  □ 配置基礎模型載入路徑")
print("  □ 配置 adapter 載入路徑")
print("  □ 設定推理參數 (batch_size, max_length 等)")
print("  □ 配置記憶體管理策略")
print("  □ 設定監控和日誌")
print("")
print("✅ 部署後驗證:")
print("  □ 推理延遲測試")
print("  □ 吞吐量測試")
print("  □ 記憶體使用監控")
print("  □ 輸出質量抽樣檢查")
print("  □ 壓力測試")
print("  □ 長時間穩定性測試")
print("")
print("📚 文檔與維護:")
print("  □ 保存部署配置文件")
print("  □ 記錄性能基準數據")
print("  □ 準備回滾方案")
print("  □ 更新 API 文檔")
print("  □ 設定告警閾值")

### 9.2 場景選擇建議

In [ ]:
print("=== Prefix Tuning 適用場景建議 ===")
print("")
print("🎯 最適合的場景:")
print("  • 需要支持多個相關任務")
print("  • 任務需要頻繁切換")
print("  • 記憶體資源有限,無法部署多個完整模型")
print("  • 文本生成任務 (對話、摘要、風格轉換)")
print("  • 可以接受適度的推理開銷")
print("")
print("⚠️  需要權衡的場景:")
print("  • 對推理延遲要求極高的場景 -> 考慮 LoRA/IA³")
print("  • 單一任務部署 -> LoRA/IA³ 更合適")
print("  • 理解類任務 (分類、NER) -> Adapter/BitFit 可能更好")
print("")
print("🔄 組合策略:")
print("  • Prefix Tuning + LoRA: 結合兩者優勢")
print("  • 開發階段用 Prefix Tuning 快速驗證多任務")
print("  • 生產階段為高頻任務單獨訓練 LoRA 部署")
print("  • 低頻任務保持 Prefix Tuning 靈活性")

## 10. 總結

通過本實驗,我們深入探討了 **Prefix Tuning 在生產環境中的部署策略**。

### 10.1 核心學習收穫

1. **理論認知**: 理解了 Prefix Tuning 不可合併的技術原因
2. **部署實踐**: 掌握了 Prefix Tuning 的保存、載入和部署流程
3. **性能分析**: 量化了推理開銷和記憶體佔用
4. **架構設計**: 學習了多任務 Prefix 管理策略
5. **方法對比**: 理解了不同 PEFT 方法的部署特性差異

### 10.2 Prefix Tuning 部署的獨特價值

- **多任務友好**: 共享基礎模型,靈活切換任務
- **記憶體效率**: Adapter 極小,多任務也不會佔用太多額外記憶體
- **部署靈活性**: 可以動態添加新任務而不影響現有服務
- **理論優雅性**: 通過前綴引導生成,概念清晰

### 10.3 部署權衡

雖然 Prefix Tuning 無法像 LoRA/IA³ 那樣實現零開銷推理,但在**多任務場景**下,其靈活性和記憶體效率使其成為優秀的選擇。關鍵是根據實際需求選擇合適的 PEFT 方法:

- **單任務、追求極致性能**: LoRA/IA³
- **多任務、靈活切換**: Prefix Tuning
- **理解類任務**: Adapter/BitFit
- **超大模型、極簡適配**: Prompt Tuning

正確的部署策略能夠最大化 PEFT 技術的價值!

In [ ]:
# 清理資源
print("=== 清理實驗資源 ===")
del peft_model, base_model, reloaded_model, fresh_base_model
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("✅ 資源清理完成")

print("\n🎉 Lab 4 - Prefix Tuning 部署指南實驗完成!")
print(f"Adapter 已保存到: {save_path}")
print("現在您可以在生產環境中部署這個 Prefix Tuning adapter 了!")